<a href="https://colab.research.google.com/github/Beatriz-Yun/AIFFEL_LMS/blob/main/Exploration/%5BE-06%5DKaggle%EC%A7%91%EA%B0%92%EC%98%88%EC%B8%A1%EA%B2%BD%EC%A7%84%EB%8C%80%ED%9A%8C_%ED%94%84%EB%A1%9C%EC%A0%9D%ED%8A%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [캐글 코리아와 함께하는 2nd ML 대회 - House Price Prediction](https://www.kaggle.com/c/2019-2nd-ml-month-with-kakr/overview)

모델의 성능을 최대화하기 위해서는 이전에 진행한 하이퍼파라미터 튜닝 뿐아니라<br>
**EDA과정을 통한 feature engineering**으로 데이터를 정제하는 것도 매우 중요하다.

<br>

튜닝은 다양한 최적화 방법론 중 하나라는 것을 알아둬야 한다.

# 목차 
### 1. 데이터 탐색
    - 문제 정의
    - 변수 설명
    - 시각화 
    - 이상치 및 결측치 확인
    - 유니크 갯수
### 2. 데이터 전처리
    - 이상치 및 결측치 처리
    - 정규화
### 3. 변수 생성
### 4. 모델링 
    - 회귀
    - GBM ( LGB/XGB )
    - 앙생블 
### 5. 잔차분석 

<br><br>

---



# 1.데이터 탐색

## 1.1 문제 정의 
이번 데이터의 목적은 집의 정보를 가지고 가격을 예측하는 회귀문제 입니다. 

평가로는 RMSE를 사용합니다. 

In [ ]:
df_train = pd.read_csv('/content/drive/MyDrive/data/train.csv')
df_test  = pd.read_csv('/content/drive/MyDrive/data/test.csv')

print("train.csv. Shape: ",df_train.shape)
print("test.csv. Shape: ",df_test.shape)

## 1.2 변수 정의

**Data fields**
1. ID : 집을 구분하는 번호
2. date : 집을 구매한 날짜
3. price : 집의 가격(Target variable)
4. bedrooms : 침실의 수
5. bathrooms : 화장실의 수
6. sqft_living : 주거 공간의 평방 피트(면적)
7. sqft_lot : 부지의 평방 피트(면적)
8. floors : 집의 층 수
9. waterfront : 집의 전방에 강이 흐르는지 유무 (a.k.a. 리버뷰)
10. view : 집이 얼마나 좋아 보이는지의 정도
11. condition : 집의 전반적인 상태
12. grade : King County grading 시스템 기준으로 매긴 집의 등급
13. sqft_above : 지하실을 제외한 평방 피트(면적)
14. sqft_basement : 지하실의 평방 피트(면적)
15. yr_built : 지어진 년도
16. yr_renovated : 집을 재건축한 년도
17. zipcode : 우편번호
18. lat : 위도
19. long : 경도
20. sqft_living15 : 2015년 기준 주거 공간의 평방 피트(면적, 집을 재건축했다면, 변화가 있을 수 있음)
21. sqft_lot15 : 2015년 기준 부지의 평방 피트(면적, 집을 재건축했다면, 변화가 있을 수 있음)

👉 우리가 예측해야 되는 값은 'price'이다.

## 1.3 시각화

## 이상치 및 결측치 확인

# 2.데이터 전처리

## 2.1 이상치 및 결측치 처리

## 2.2 정규화

# 3.변수 생성

#4. 모델링

## ✓ 튜닝해볼 수 있는 모델 클래스 인자

대표적으로 자주 튜닝하는 lightgbm 라이브러리의 인자는 다음과 같다.

- max_depth : 의사 결정 나무의 깊이, 정수 사용
- learning_rate : 한 스텝에 이동하는 양을 결정하는 파라미터, 보통 0.0001\~0.1 사이의 실수 사용
- n_estimators : 사용하는 개별 모델의 개수, 보통 50~100 이상의 정수 사용
- num_leaves : 하나의 LightGBM 트리가 가질 수 있는 최대 잎의 수
- boosting_type : 부스팅 방식, gbdt, rf 등의 문자열 입력

# 프로젝트 루브릭 (평가 기준)
- 데이터 전처리, 모델학습, 예측의 전체 과정을 거쳐 캐글 submission까지 전과정이 성공적으로 진행되었는가?
- 제출된 노트북이 캐글 커널로 사용될 수 있을 만큼 전처리, 학습, 최적화 진행 과정이 체계적으로 기술되었는가?
- 다양한 피처 엔지니어링과 하이퍼 파라미터 튜닝 등의 최적화 기법을 통해 캐글 리더보드의 Private score 기준 110000 이하의 점수를 얻었는가?